# Packaging tests

In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument #
import json #
import multiprocessing#

# import joblib 
# import pandas as pd
# from nltk.corpus import stopwords 
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# import joblib
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# from pprint import pprint
# import math




In [3]:
df = joblib.load('../../../raw_data/processed_data.joblib')
df['tag_language'] = df['tag_language'].fillna(value='en')

In [4]:
# select english jobs
df_eng = df.copy()
df_eng = df_eng[df_eng['tag_language'] == 'en']
df_eng.reset_index(inplace=True)
df_eng.drop(columns='index', inplace=True)

In [5]:
# join strings
def join_strings(text):
    return ' '.join(text)

# lemmatize
def lemmatize_words(word):
    lemmatizer = WordNetLemmatizer()
    lemmatized = lemmatizer.lemmatize(word)

    return lemmatized

# remove stopwords
def remove_stopwords(text):

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text) 
    text = [w for w in word_tokens if not w in stop_words] 
  
    return text

In [6]:
# process text
df_eng['clean'] = df_eng['job_text_tokenized'].apply(join_strings).apply(lemmatize_words)\
    .apply(remove_stopwords)


# functions for pipeline (ignore code above)

**Workflow:**

1. load the model if present. if not, instanciate model
2. add a tag to all job offers
3. train model on tagged texts (in the saved model I trained it on 3000 datapoints, 15 epochs)
4. save the model (if model not present)
5. find similar job offers:
    get recomendation based on copy-pasted offer or get recomendation based on ???? (decide this); 
    output of top 10 (or another number) similar to input offer

In [46]:
def load_Doc2Vec(path_to_files):
    '''loads and returns pre-trained model'''
    model_dbow = Doc2Vec.load(path_to_files)
    
    return model_dbow

In [26]:
def instanciate_Doc2Vec(text_tagged):
    '''instanciates model, using dbow (d=0)'''
    cores = multiprocessing.cpu_count()
    model_dbow = Doc2Vec(documents=text_tagged,
                         dm=0,
                         alpha=0.025,
                         vector_size=len(text_tagged), 
                         min_count=1,
                         workers=cores,
                        )

In [11]:
def tag_texts(texts):
    ''' input: pandas series of datasets to use in the model
        returns texts with identification tag
    '''
    texts_tagged = [TaggedDocument(text, tags=['tag_'+str(tag)]) for tag, text in enumerate(texts)]

    return texts_tagged

texts_tagged = tag_texts(texts)
#texts_tagged[0]

In [27]:
def train_Doc2Vec(texts_tagged_small):
    ''' trains model'''
    model_dbow.train(text_tagged,
                     total_examples=model_dbow.corpus_count, 
                     epochs=15,
                    )

In [ ]:
def save_Doc2Vec(path):
    '''saves trained Doc2Vec
    input: path to location 
    '''
    model_dbow.save(path)

In [54]:
# infer vector and get similarity
def find_similar_jobs(model, tokenized_offers, number_offers=10):
    ''' inputs: Doc2Vec model, string of a tokenized job offer, number of offers. 
        returns: tags of top x most similar job offers and similarity probabilities
    '''
    # infer vector from text 
    infer_vector = model.infer_vector(tokenized_offers)
    # find similar offers
    similar_documents = model.docvecs.most_similar([infer_vector], topn = number_offers)

    return similar_documents

In [24]:
texts_tagged_small = texts_tagged[:3000]

In [55]:
model = load_Doc2Vec('../../../models/doc2vec_3000_15_epochs')

In [58]:
find_similar_jobs(model, texts_tagged_small[0][0], 10)

[('tag_272', 0.9891966581344604),
 ('tag_754', 0.9883853197097778),
 ('tag_0', 0.9864751100540161),
 ('tag_654', 0.7715011835098267),
 ('tag_368', 0.7501724362373352),
 ('tag_382', 0.7268041372299194),
 ('tag_100', 0.6558924913406372),
 ('tag_723', 0.6515829563140869),
 ('tag_65', 0.6219576597213745),
 ('tag_111', 0.6144018173217773)]

## start pipeline

In [37]:
class Doc2Vec_Pipeline:
    
    # instanciate class
    def __init__(self, path=None, texts=None):
        if path:
            self.d2v_model = Doc2Vec.load(path)
        else:
            cores = multiprocessing.cpu_count()
            self.d2v_model = Doc2Vec(documents=df,
                     dm=0,
                     alpha=0.025,
                     vector_size=len(df), 
                     min_count=1,
                     workers=cores)
    
    # build vocabulary
    def tag_docs(self, texts):
        ''' input: pandas series of datasets to use in the model
            returns texts with identification tag
        '''
        self.texts_tagged = [TaggedDocument(text, tags=['tag_'+str(tag)]) for tag, text in enumerate(texts)]

        return self.texts_tagged


    
    
    # train model
    
    # define most similar jobs
    
    # 